# Beta-VAE training & evaluation demo
This notebook builds a tiny synthetic dataset, trains the provided Beta-VAE implementation, and runs the evaluation helpers on the saved checkpoint.

In [1]:
import os, sys, pathlib, yaml
ROOT = pathlib.Path.cwd()
sys.path.insert(0, str(ROOT / 'src'))
os.environ['PYTHONPATH'] = str(ROOT / 'src')

base_cfg_path = ROOT / 'configs' / 'beta_vae_se.yaml'
with open(base_cfg_path) as f:
    cfg = yaml.safe_load(f)

demo_outputs = ROOT / 'outputs' / 'demo_run'
cfg['paths']['processed_dir'] = str(ROOT / 'data' / 'demo_processed')
cfg['paths']['outputs_dir'] = str(demo_outputs)
cfg['paths']['models_dir'] = str(demo_outputs / 'models')
cfg['paths']['figures_dir'] = str(demo_outputs / 'figures')
cfg['paths']['tables_dir'] = str(demo_outputs / 'tables')
cfg['paths']['run_id'] = 'notebook_demo'

cfg['training']['batch_size'] = 4
cfg['training']['epochs'] = 3
cfg['debug']['enabled'] = True
cfg['debug']['epochs'] = 3
cfg['debug']['train_samples'] = 24
cfg['debug']['test_samples'] = 12
cfg['debug']['max_train_batches'] = 6
cfg['debug']['max_val_batches'] = 6
cfg['evaluation']['probe_train_split'] = 0.6

demo_cfg_path = ROOT / 'configs' / 'demo_notebook.yaml'
demo_cfg_path.parent.mkdir(exist_ok=True)
with open(demo_cfg_path, 'w') as f:
    yaml.safe_dump(cfg, f)
os.environ['CONFIG_PATH'] = str(demo_cfg_path)
print('Using config:', demo_cfg_path)


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\jason\\VSProjects\\Beta-VAE\\notebooks\\configs\\beta_vae_se.yaml'

In [ ]:
from pathlib import Path
import numpy as np
from PIL import Image
import yaml, os

with open(os.environ['CONFIG_PATH']) as f:
    cfg = yaml.safe_load(f)
proc_root = Path(cfg['paths']['processed_dir'])
size = cfg['data']['image_size']
classes = ['glioma', 'meningioma', 'notumor']
split_counts = {'train': 24, 'test': 12}
rng = np.random.default_rng(0)

def make_canvas(base_level=0.3, noise=0.05):
    arr = rng.normal(base_level, noise, size=(size, size))
    return np.clip(arr, 0, 1)

def pattern_for_class(cls):
    arr = make_canvas(0.25, 0.08)
    yy, xx = np.mgrid[:size, :size]
    if cls == 'glioma':
        circle = (xx - size//2)**2 + (yy - size//2)**2 <= (size//4)**2
        arr[circle] += 0.35
    elif cls == 'meningioma':
        band = (yy % (size//8)) < (size//16)
        arr[band] += 0.25
    else:  # notumor
        arr += rng.normal(0.0, 0.02, size=arr.shape)
    return np.clip(arr, 0, 1)

for split, total in split_counts.items():
    per_class = total // len(classes)
    for cls in classes:
        out_dir = proc_root / split / cls
        out_dir.mkdir(parents=True, exist_ok=True)
        for idx in range(per_class):
            arr = pattern_for_class(cls)
            img = Image.fromarray((arr * 255).astype(np.uint8), mode='L')
            img.save(out_dir / f"{cls}_{idx}.png")
print('Synthetic dataset written to', proc_root)


In [ ]:
from training.train import train
train()

In [ ]:
import torch
from utils.brain_tumor_utils.config_parser import get_config
from utils.brain_tumor_utils.datautils import build_dataloaders
from data_processing.augmentations import get_train_transforms, get_test_transforms
from evaluation.run_evaluation import load_model
from evaluation.recon_metrics import evaluate_full

cfg = get_config()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, test_loader = build_dataloaders(
    transform_train=get_train_transforms(),
    transform_test=get_test_transforms()
)
model = load_model('best')
model.to(device)
metrics = evaluate_full(model, train_loader, test_loader, device)
metrics